In [9]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report, confusion_matrix

# Entraînement du modèle One-Class SVM
svm_model = OneClassSVM(kernel='rbf', gamma='auto', nu=0.05)
svm_model.fit(X_scaled)

# Prédictions
y_pred_svm = svm_model.predict(X_scaled)

# Conversion en étiquettes compatibles (1=normal, -1=anomalie vers 0=normal, 1=anomalie)
df['anomaly_svm'] = y_pred_svm
df['anomaly_svm'] = df['anomaly_svm'].map({1: 0, -1: 1}) # 0=normal, 1=anomalie

# Évaluation : attention, on compare avec anomaly_type != NaN (existe seulement pour les anomalies injectées)
true_labels = df['anomaly_type'].notna().astype(int)

print("📊 Rapport de classification pour One-Class SVM :")
print(classification_report(true_labels, df['anomaly_svm'], target_names=["Normal", "Anomalie"]))

# Matrice de confusion
print("📊 Matrice de confusion :")
print(confusion_matrix(true_labels, df['anomaly_svm']))

📊 Rapport de classification pour One-Class SVM :
              precision    recall  f1-score   support

      Normal       0.97      0.88      0.92       955
    Anomalie       0.14      0.42      0.21        45

    accuracy                           0.86      1000
   macro avg       0.56      0.65      0.57      1000
weighted avg       0.93      0.86      0.89      1000

📊 Matrice de confusion :
[[841 114]
 [ 26  19]]


In [11]:
import pandas as pd

# ... (votre code de chargement, preprocessing, entraînement One-Class SVM et évaluation) ...

# Récupérer les prédictions de la colonne 'anomaly_svm' du DataFrame
svm_predictions = df['anomaly_svm'].values

# Convertir les vraies étiquettes originales (df['log_type']) en numérique (0=normal, 1=anomalie)
true_labels_numeric = [1 if label == 'anomaly' else 0 for label in df['log_type']]

# Créer un DataFrame pour les résultats du One-Class SVM
svm_results_df = pd.DataFrame({'prediction': svm_predictions, 'true_label': true_labels_numeric})

# Sauvegarder les résultats dans un fichier CSV dans le dossier 'results'
svm_results_df.to_csv("../results/oneclasssvm_predictions.csv", index=False)

print("Les prédictions du One-Class SVM ont été sauvegardées dans '../results/oneclasssvm_predictions.csv'")

Les prédictions du One-Class SVM ont été sauvegardées dans '../results/oneclasssvm_predictions.csv'


In [8]:
svm_model.fit(X_scaled)

OneClassSVM(gamma='auto', nu=0.05)

In [7]:
from sklearn.svm import OneClassSVM

# Entraînement du modèle One-Class SVM
svm_model = OneClassSVM(kernel='rbf', gamma='auto', nu=0.05)
svm_model.fit(X_scaled)

# Le reste de votre code pour les prédictions et l'évaluation suivra.

OneClassSVM(gamma='auto', nu=0.05)

In [6]:
scaler = StandardScaler(with_mean=False)
X_scaled = scaler.fit_transform(X_processed)

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=True), categorical_features)
    ],
    remainder='drop'
)

df_model = df.copy()
X = df_model.drop(columns=["timestamp", "ip", "user", "log_type", "anomaly_type"])
X_processed = preprocessor.fit_transform(X)

In [4]:
df_model = df.copy()
X = df_model.drop(columns=["timestamp", "ip", "user", "log_type", "anomaly_type"])

In [3]:
df["timestamp"] = pd.to_datetime(df["timestamp"])
df["hour"] = df["timestamp"].dt.hour
df["day_of_week"] = df["timestamp"].dt.day_name()

def get_time_period(hour):
    if 5 <= hour < 12:
        return "morning"
    elif 12 <= hour < 17:
        return "afternoon"
    elif 17 <= hour < 21:
        return "evening"
    else:
        return "night"
df["time_period"] = df["hour"].apply(get_time_period)

def classify_ip(ip):
    if ip.startswith("192.") or ip.startswith("10.") or ip.startswith("172."):
        return "local"
    else:
        return "public"
df["ip_type"] = df["ip"].apply(classify_ip)

def classify_status(status):
    if 200 <= status < 300:
        return "2xx_success"
    elif 400 <= status < 500:
        return "4xx_client_error"
    elif 500 <= status < 600:
        return "5xx_server_error"
    else:
        return "other"
df["status_class"] = df["status"].apply(classify_status)

In [2]:
categorical_features = ["method", "endpoint", "time_period", "ip_type", "status_class", "day_of_week"]
numeric_features = ["hour"]

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
# Définissez le chemin vers votre fichier CSV
log_file_path = os.path.join("..", "data", "generated_logs.csv")
df = pd.read_csv(log_file_path)

# Si vous avez les étapes de création des features temporelles/IP/statut, exécutez-les aussi
df["timestamp"] = pd.to_datetime(df["timestamp"])
df["hour"] = df["timestamp"].dt.hour
df["day_of_week"] = df["timestamp"].dt.day_name()
# ... et ainsi de suite pour les autres features